In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score


# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer#for word embedding
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /home/francesco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/francesco/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/francesco/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/francesco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
data = pd.read_csv('network_tweets.csv')
# check dtypes
data.info()

/tmp/ipykernel_78326/1248792684.py:1: DtypeWarning: Columns (0,1,5,11,16,17,24) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('network_tweets.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394355 entries, 0 to 394354
Data columns (total 29 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               394355 non-null  object 
 1   userid                   394329 non-null  object 
 2   username                 394329 non-null  object 
 3   acctdesc                 359051 non-null  object 
 4   location                 254714 non-null  object 
 5   following                394329 non-null  object 
 6   followers                394329 non-null  float64
 7   totaltweets              394329 non-null  float64
 8   usercreatedts            394313 non-null  object 
 9   tweetid                  394329 non-null  float64
 10  tweetcreatedts           394329 non-null  object 
 11  retweetcount             394326 non-null  object 
 12  text                     394329 non-null  object 
 13  hashtags                 394310 non-null  object 
 14  lang

In [15]:
data.head()

,Unnamed: 0,userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetid,...,original_tweet_userid,original_tweet_username,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,is_quote_status,quoted_status_id,quoted_status_userid,quoted_status_username,extractedts
0,6,77538832,bn880,"Putin has made a huge mistake, he is about to ...",Canada,116,210.0,13236.0,2009-09-26 18:22:00,1.520916e+18,...,1.370616e+18,Blue_Sauron,0.000000e+00,0.000000e+00,NaN,False,0.0,0.0,NaN,2022-05-02 00:25:56.066862
1,33,14027452,hardknoxfirst,"Liberal, supports law enforcement, guitarist, ...","Columbus native, SE Ohio now.",5586,6826.0,559945.0,2008-02-26 21:00:11,1.520916e+18,...,0.000000e+00,NaN,0.000000e+00,0.000000e+00,NaN,False,0.0,0.0,NaN,2022-05-02 00:28:46.193536
2,40,1438291651135344640,wediyalula,Truth things,NaN,4917,4779.0,81263.0,2021-09-16 00:00:40,1.520916e+18,...,1.503409e+18,TigrayKwgehEu,0.000000e+00,0.000000e+00,NaN,False,0.0,0.0,NaN,2022-05-02 00:25:55.095644
3,50,969040753,FreedomHonor666,Let's talk truly about any struggle in the wor...,Universe,1468,854.0,6603.0,2012-11-25 00:03:01,1.520916e+18,...,2.758100e+09,GlasnostGone,0.000000e+00,0.000000e+00,NaN,False,0.0,0.0,NaN,2022-05-02 00:25:54.986379
4,53,1832502638,eveemars87,Can´t stand injustices #SlavaUkraini #ArmUkra...,lisboa,204,245.0,43849.0,2013-09-08 18:02:23,1.520916e+18,...,0.000000e+00,NaN,1.520499e+18,2.659312e+09,irmi449,False,0.0,0.0,NaN,2022-05-02 05:14:31.916005


In [16]:
data = data[['tweetid', 'tweetcreatedts', 'language', 'location', 'text', 'hashtags']]
data = data[data['language'] == 'en']

# data = data.head(10000)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 394291 entries, 0 to 394354
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tweetid         394291 non-null  float64
 1   tweetcreatedts  394291 non-null  object 
 2   language        394291 non-null  object 
 3   location        254678 non-null  object 
 4   text            394291 non-null  object 
 5   hashtags        394291 non-null  object 
dtypes: float64(1), object(5)
memory usage: 21.1+ MB


In [17]:
# import these modules
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()

#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'&', 'and', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r't\.co\S+', ' ', text)
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    text = ps.stem(text)
    return text
 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)


In [18]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
    
data['clean_text'] = data['text'].apply(lambda x: finalpreprocess(x))
data.head()

,tweetid,tweetcreatedts,language,location,text,hashtags,clean_text
0,1.520916e+18,2022-05-02 00:00:01,en,Canada,"📸 Russian MANPADS shot missed its target, pict...",[],russian manpad shot miss target pictured ukrai...
1,1.520916e+18,2022-05-02 00:00:06,en,"Columbus native, SE Ohio now.","No, no, Noam &gt; Chomsky Calls Trump the 'One...","[{'text': 'Ukraine', 'indices': [111, 119]}]",noam andgt chomsky call trump one western stat...
2,1.520916e+18,2022-05-02 00:00:09,en,NaN,The absence of compassion from the Int'l Commu...,"[{'text': 'Tigray', 'indices': [95, 102]}]",absence compassion int l community towards suf...
3,1.520916e+18,2022-05-02 00:00:11,en,Universe,R.I.P Scott and thank you. A British national ...,"[{'text': 'Ukraine', 'indices': [83, 91]}]",r p scott thank british national kill ukraine ...
4,1.520916e+18,2022-05-02 00:00:11,en,lisboa,@irmi449 @OksanaUkka 💔💔😢😢Russia stole their in...,"[{'text': 'SlavaUkraini', 'indices': [75, 88]}]",irmi oksanaukka russia stole innocence pay sla...


In [19]:
data.to_parquet('testset.parquet.gzip', compression='gzip')

/home/francesco/anaconda3/envs/NLP3/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
